In [1]:
import tensorflow as tf 
import pandas as pd
import numpy as np
import os

In [11]:
import tensorflow as tf 
import pandas as pd
import numpy as np
import os

input_x_size=20
field_size=2

##s 此处维度为3 ？ 
vector_dimension=3

total_plan_train_steps=1000
# 使用SGD 随机梯度下降法 ，每一个样本进行依次梯度下降，更新参数
batch_size=1
all_data_size=1000

lr=0.01
MODEL_SAVE_PATH="TFModel_self"
MODEL_NAME= "FFM"

def createTwoDimensionWeight(input_x_size,field_size,vector_dimension):
    weights=tf.truncated_normal([input_x_size,field_size,vector_dimension])
    tf_weights=tf.Variable(weights)
    return tf_weights

def createOneDimensionWeight(input_x_size):
    weights=tf.truncated_normal([input_x_size])
    tf_weights=tf.Variable(weights)
    return tf_weights

def createZeroDimensionWeight():
    weights=tf.truncated_normal([1])
    tf_weights=tf.Variable(weights)
    return tf_weights

def inference(input_x,input_x_field,zeroWeights,oneDimWeights,thirdWeight):
    """ 计算回归模型输出的值"""
    secondValue=tf.reduce_sum(tf.multiply(oneDimWeights,input_x,name='secondValue'))
    firstTwoValue=tf.add(zeroWeights,secondValue,name="firstTwoValue")
    
    thirdValue=tf.Variable(0.0,dtype=tf.float32)
    input_shape=input_x_size
    
    for i in range(input_shape):
        featureIndex1=i
        fieldIndex1 = int(input_x_field[i])
        for j in range(i+1,input_shape):
            featureIndex2=j
            fieldIndex2=int(input_x_field[j])
            ## 下面这个地方具体是指  ？
            vectorLeft=tf.convert_to_tensor([[featureIndex1,fieldIndex2,i] for i in range(vector_dimension)])
            weightLeft=tf.gather_nd(thirdWeight,vectorLeft)
            #  gather_nd(params, indices, name=None)实现了根据指定的 参数 indices 来提取params 的元素重建出一个tensor,
            weightLeftAfterCut=tf.squeeze(weightLeft)
            # tf.squeeze() 给定张量输入，此操作返回相同类型的张量，并删除所有尺寸为1的尺寸。 如果不想删除所有尺寸1尺寸，可以通过指定squeeze_dims来删除特定尺寸1尺寸。
            
            vectorRight=tf.convert_to_tensor([[featureIndex2,fieldIndex1,i] for i in range(vector_dimension)])
            weightRight=tf.gather_nd(thirdWeight,vectorRight)
            weightRightAfterCut=tf.squeeze(weightRight)
            
            tempValue=tf.reduce_sum(tf.multiply(weightLeftAfterCut,weightRightAfterCut))
            
## todo  带请教，上面的vectorLeft、weightLeft、weightRightAfterCut、tempValue 等都是对应FFM公式里的那部分？
            
            indices2=[i]
            indices3=[j]
            
            xi=tf.squeeze(tf.gather_nd(input_x,indices2))
            xj=tf.squeeze(tf.gather_nd(input_x,indices3))
            
            product=tf.reduce_sum(tf.multiply(xi,xj))
            
            secondItemVal=tf.multiply(tempValue,product)
            
            tf.assign(thirdValue,tf.add(thirdValue,secondItemVal))
            # tf.assign(A, new_number): 这个函数的功能主要是把A的值变为new_number
            
    return tf.add(firstTwoValue,thirdValue)
            
            
            
            
            
        

In [12]:
## 生成数据
def gen_data():
    labels=[-1,1]
    y=[np.random.choice(labels,1)[0] for _ in range(all_data_size)]
    # 上面np.random.choice(a, size=3, replace=False, p=None) 的含义是随机从a只能跟选择出size个，如果P没有指定的时候相当于是一致分布。
    x_field=[i//10 for i in range(input_x_size)]
    # input_x_size = 20 ,数据涉及20维特征，前10维特征是一个field，后十维是一个field
    x =np.random.randint(0,2,size=(all_data_size,input_x_size))
    return x,y ,x_field
    
    

In [15]:
if __name__=='__main__':
    global_step=tf.Variable(0,trainable=False)
    trainx,trainy,train_field=gen_data()
    
    # 
    input_x=tf.placeholder(tf.float32,[input_x_size])
    input_y=tf.placeholder(tf.float32)
    
    #
    lambda_w=tf.constant(0.001,name='lambda_w')
    lambda_v=tf.constant(0.001,name='lambda_v')
    
    zeroWeights=createZeroDimensionWeight()
    oneDimWeights=createOneDimensionWeight(input_x_size)  # 创建二次项的权重变量
    thirdWeight=createTwoDimensionWeight(input_x_size,
                                        field_size,
                                        vector_dimension)  # n * f * k
    y_=inference(input_x,train_field,zeroWeights,oneDimWeights,thirdWeight)
    
    l2_norm=tf.reduce_sum(
        tf.add(
            tf.multiply(lambda_w,tf.pow(oneDimWeights,2)),
            tf.reduce_sum(tf.multiply(lambda_v,tf.pow(thirdWeight,2)),axis=[1,2])          
        )
    )
    
    loss=tf.log(1+tf.exp(-input_y*y_))+l2_norm
    
    train_step=tf.train.GradientDescentOptimizer(learning_rate=lr).minimize(loss)
    
    saver=tf.train.Saver()
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for i in range(total_plan_train_steps):
            for t in range(all_data_size):   
                input_x_batch=trainx[t]
                input_y_batch=trainy[t]
                predict_loss,_,steps=sess.run([loss,train_step,global_step],
                                             feed_dict={input_x:input_x_batch,input_y:input_x_batch})

                print(" After {step} training  step(s), loss on training batch is {predict_loss}".format(step=steps,predict_loss=predict_loss))
                saver.save(sess,os.path.join(MODEL_SAVE_PATH,MODEL_NAME),global_step=steps)
                writer=tf.summary.FileWriter(os.path.join(MODEL_SAVE_PATH,MODEL_NAME),tf.get_default_graph())
                writer.close()


 After 0 training  step(s), loss on training batch is [1.1331886 1.1331886 1.1331886 0.7741379 1.1331886 0.7741379 0.7741379
 1.1331886 0.7741379 1.1331886 1.1331886 1.1331886 0.7741379 1.1331886
 1.1331886 1.1331886 1.1331886 0.7741379 1.1331886 1.1331886]
 After 0 training  step(s), loss on training batch is [0.64809656 0.64809656 0.64809656 0.77429307 0.64809656 0.77429307
 0.77429307 0.77429307 0.77429307 0.64809656 0.64809656 0.64809656
 0.64809656 0.64809656 0.64809656 0.64809656 0.77429307 0.64809656
 0.77429307 0.77429307]
 After 0 training  step(s), loss on training batch is [0.33300203 0.7743935  0.7743935  0.33300203 0.7743935  0.7743935
 0.7743935  0.7743935  0.7743935  0.33300203 0.7743935  0.33300203
 0.7743935  0.33300203 0.33300203 0.7743935  0.7743935  0.33300203
 0.7743935  0.7743935 ]
 After 0 training  step(s), loss on training batch is [0.7743974 0.7743974 0.7743974 1.4744775 0.7743974 0.7743974 0.7743974
 0.7743974 1.4744775 0.7743974 0.7743974 0.7743974 0.7743974

 After 0 training  step(s), loss on training batch is [0.09188844 0.77350944 0.09188844 0.09188844 0.09188844 0.09188844
 0.77350944 0.09188844 0.77350944 0.09188844 0.77350944 0.77350944
 0.09188844 0.09188844 0.77350944 0.77350944 0.77350944 0.77350944
 0.09188844 0.09188844]
 After 0 training  step(s), loss on training batch is [0.08554959 0.77345616 0.08554959 0.77345616 0.08554959 0.77345616
 0.08554959 0.77345616 0.08554959 0.77345616 0.08554959 0.08554959
 0.08554959 0.08554959 0.77345616 0.77345616 0.77345616 0.08554959
 0.08554959 0.08554959]
 After 0 training  step(s), loss on training batch is [0.77339834 0.77339834 0.77339834 0.77339834 0.77339834 0.35347486
 0.35347486 0.77339834 0.35347486 0.35347486 0.35347486 0.35347486
 0.35347486 0.35347486 0.35347486 0.35347486 0.77339834 0.77339834
 0.77339834 0.77339834]
 After 0 training  step(s), loss on training batch is [0.77338994 0.77338994 0.77338994 0.10895887 0.10895887 0.77338994
 0.77338994 0.10895887 0.10895887 0.773389

 After 0 training  step(s), loss on training batch is [0.08034176 0.08034176 0.08034176 0.08034176 0.08034176 0.08034176
 0.08034176 0.77217865 0.77217865 0.08034176 0.77217865 0.08034176
 0.77217865 0.77217865 0.08034176 0.08034176 0.08034176 0.77217865
 0.08034176 0.08034176]
 After 0 training  step(s), loss on training batch is [0.77211773 0.77211773 0.77211773 0.77211773 0.77211773 0.77211773
 0.77211773 0.21612698 0.21612698 0.21612698 0.21612698 0.77211773
 0.21612698 0.77211773 0.21612698 0.21612698 0.21612698 0.77211773
 0.77211773 0.77211773]
 After 0 training  step(s), loss on training batch is [0.08522926 0.08522926 0.772087   0.08522926 0.772087   0.08522926
 0.08522926 0.08522926 0.08522926 0.08522926 0.08522926 0.08522926
 0.08522926 0.08522926 0.08522926 0.08522926 0.772087   0.08522926
 0.772087   0.772087  ]
 After 0 training  step(s), loss on training batch is [0.7720327  0.10260367 0.7720327  0.10260367 0.7720327  0.10260367
 0.7720327  0.10260367 0.10260367 0.772032

 After 0 training  step(s), loss on training batch is [0.7706286 0.3645135 0.3645135 0.7706286 0.7706286 0.3645135 0.7706286
 0.3645135 0.3645135 0.3645135 0.7706286 0.3645135 0.3645135 0.7706286
 0.7706286 0.3645135 0.7706286 0.7706286 0.7706286 0.7706286]
 After 0 training  step(s), loss on training batch is [0.12195626 0.12195626 0.12195626 0.770598   0.770598   0.12195626
 0.770598   0.770598   0.770598   0.770598   0.770598   0.770598
 0.770598   0.770598   0.770598   0.770598   0.12195626 0.770598
 0.770598   0.770598  ]
 After 0 training  step(s), loss on training batch is [0.08857373 0.08857373 0.08857373 0.08857373 0.77054715 0.08857373
 0.77054715 0.08857373 0.08857373 0.08857373 0.77054715 0.08857373
 0.08857373 0.08857373 0.77054715 0.77054715 0.77054715 0.77054715
 0.77054715 0.08857373]
 After 0 training  step(s), loss on training batch is [0.07863318 0.7704958  0.7704958  0.07863318 0.07863318 0.07863318
 0.07863318 0.7704958  0.7704958  0.07863318 0.07863318 0.7704958
 

 After 0 training  step(s), loss on training batch is [0.11521181 0.11521181 0.76900196 0.11521181 0.76900196 0.76900196
 0.76900196 0.76900196 0.76900196 0.11521181 0.11521181 0.76900196
 0.76900196 0.11521181 0.76900196 0.11521181 0.76900196 0.76900196
 0.76900196 0.76900196]
 After 0 training  step(s), loss on training batch is [0.7689554  0.7689554  0.7689554  0.7689554  0.7689554  0.7689554
 0.7689554  0.7689554  0.7689554  0.7689554  0.7689554  0.7689554
 0.12783967 0.7689554  0.12783967 0.7689554  0.12783967 0.7689554
 0.12783967 0.7689554 ]
 After 0 training  step(s), loss on training batch is [0.0809743 0.0809743 0.0809743 0.7689042 0.0809743 0.0809743 0.7689042
 0.7689042 0.7689042 0.7689042 0.7689042 0.0809743 0.7689042 0.7689042
 0.7689042 0.0809743 0.0809743 0.0809743 0.7689042 0.7689042]
 After 0 training  step(s), loss on training batch is [0.76884794 0.07887875 0.76884794 0.07887875 0.07887875 0.76884794
 0.07887875 0.07887875 0.07887875 0.76884794 0.76884794 0.76884794

...

 After 0 training  step(s), loss on training batch is [0.75408036 0.0612812  0.0612812  0.75408036 0.0612812  0.0612812
 0.0612812  0.0612812  0.75408036 0.75408036 0.75408036 0.0612812
 0.75408036 0.0612812  0.0612812  0.0612812  0.0612812  0.0612812
 0.0612812  0.75408036]
 After 0 training  step(s), loss on training batch is [0.06739613 0.75403225 0.75403225 0.75403225 0.75403225 0.06739613
 0.06739613 0.06739613 0.75403225 0.06739613 0.06739613 0.06739613
 0.06739613 0.06739613 0.75403225 0.75403225 0.75403225 0.06739613
 0.75403225 0.75403225]
 After 0 training  step(s), loss on training batch is [0.75398874 0.75398874 0.75398874 0.75398874 0.06532745 0.06532745
 0.06532745 0.75398874 0.75398874 0.06532745 0.75398874 0.75398874
 0.06532745 0.06532745 0.75398874 0.06532745 0.06532745 0.75398874
 0.06532745 0.75398874]
 After 0 training  step(s), loss on training batch is [0.06956326 0.75394356 0.06956326 0.75394356 0.75394356 0.06956326
 0.06956326 0.06956326 0.75394356 0.06956326 

 After 0 training  step(s), loss on training batch is [0.7527336  0.7527336  0.7527336  0.06068322 0.06068322 0.7527336
 0.7527336  0.06068322 0.06068322 0.7527336  0.7527336  0.7527336
 0.7527336  0.06068322 0.7527336  0.7527336  0.06068322 0.06068322
 0.06068322 0.7527336 ]
 After 0 training  step(s), loss on training batch is [0.0608747 0.0608747 0.7526869 0.0608747 0.7526869 0.0608747 0.0608747
 0.0608747 0.7526869 0.0608747 0.7526869 0.7526869 0.0608747 0.7526869
 0.0608747 0.7526869 0.0608747 0.7526869 0.0608747 0.7526869]
 After 0 training  step(s), loss on training batch is [0.7526409  0.06129242 0.7526409  0.06129242 0.06129242 0.06129242
 0.7526409  0.7526409  0.7526409  0.06129242 0.7526409  0.7526409
 0.7526409  0.7526409  0.06129242 0.7526409  0.7526409  0.06129242
 0.7526409  0.06129242]
 After 0 training  step(s), loss on training batch is [0.7525948  0.7525948  0.06959531 0.06959531 0.06959531 0.7525948
 0.7525948  0.06959531 0.06959531 0.7525948  0.06959531 0.7525948
 

 After 0 training  step(s), loss on training batch is [0.7514316  0.05861632 0.7514316  0.05861632 0.05861632 0.05861632
 0.7514316  0.7514316  0.7514316  0.7514316  0.05861632 0.7514316
 0.05861632 0.7514316  0.05861632 0.7514316  0.05861632 0.05861632
 0.05861632 0.05861632]
 After 0 training  step(s), loss on training batch is [0.75138545 0.75138545 0.09840277 0.75138545 0.75138545 0.75138545
 0.75138545 0.09840277 0.09840277 0.75138545 0.75138545 0.75138545
 0.75138545 0.09840277 0.75138545 0.09840277 0.09840277 0.75138545
 0.75138545 0.75138545]
 After 0 training  step(s), loss on training batch is [0.07552064 0.07552064 0.7513488  0.07552064 0.7513488  0.07552064
 0.7513488  0.7513488  0.7513488  0.07552064 0.7513488  0.7513488
 0.07552064 0.7513488  0.7513488  0.07552064 0.7513488  0.07552064
 0.7513488  0.7513488 ]
 After 0 training  step(s), loss on training batch is [0.75131035 0.05896212 0.05896212 0.05896212 0.05896212 0.75131035
 0.05896212 0.75131035 0.75131035 0.75131035

 After 0 training  step(s), loss on training batch is [0.06156818 0.75015116 0.06156818 0.75015116 0.75015116 0.06156818
 0.75015116 0.75015116 0.75015116 0.75015116 0.06156818 0.06156818
 0.75015116 0.75015116 0.75015116 0.75015116 0.06156818 0.06156818
 0.06156818 0.06156818]
 After 0 training  step(s), loss on training batch is [0.750109   0.05759583 0.750109   0.750109   0.05759583 0.750109
 0.05759583 0.750109   0.05759583 0.750109   0.750109   0.05759583
 0.05759583 0.05759583 0.05759583 0.05759583 0.05759583 0.05759583
 0.750109   0.750109  ]
 After 0 training  step(s), loss on training batch is [0.05704661 0.05704661 0.05704661 0.05704661 0.05704661 0.7500643
 0.05704661 0.05704661 0.7500643  0.7500643  0.7500643  0.7500643
 0.05704661 0.7500643  0.05704661 0.7500643  0.05704661 0.05704661
 0.05704661 0.7500643 ]
 After 0 training  step(s), loss on training batch is [0.7500191  0.06624888 0.7500191  0.7500191  0.7500191  0.06624888
 0.06624888 0.7500191  0.06624888 0.7500191  0

KeyboardInterrupt: 

In [2]:
temp=[[2,m] for m in range(10)]
print(temp)

[[2, 0], [2, 1], [2, 2], [2, 3], [2, 4], [2, 5], [2, 6], [2, 7], [2, 8], [2, 9]]


In [5]:
import tensorflow as tf 
a=tf.constant([[[[1],[1.1],[1.5]]],[[[3.1],[3.2],[3.3]]]])
print(a.shape)
b=tf.squeeze(a)
print(b.shape)
print(b)
with tf.Session() as sess:
    print(sess.run(b))

(2, 1, 3, 1)
(2, 3)
Tensor("Squeeze_2:0", shape=(2, 3), dtype=float32)
[[1.  1.1 1.5]
 [3.1 3.2 3.3]]


In [9]:
labels=[-1,1]
y=[np.random.choice(labels,1)[0] for _ in range(all_data_size)]
print(y)

[1, 1, 1, 1, 1, -1, 1, -1, -1, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, 1, 1, -1, 1, -1, 1, 1, -1, 1, 1, -1, -1, 1, -1, 1, -1, 1, -1, -1, -1, -1, 1, -1, 1, 1, 1, 1, 1, -1, 1, 1, -1, 1, -1, 1, 1, -1, 1, -1, -1, 1, -1, 1, -1, 1, 1, 1, -1, -1, -1, 1, 1, 1, 1, -1, -1, 1, -1, 1, -1, 1, 1, -1, 1, -1, 1, 1, 1, -1, 1, 1, -1, -1, -1, -1, 1, 1, -1, 1, 1, -1, -1, -1, 1, 1, -1, -1, -1, 1, 1, 1, 1, 1, -1, -1, -1, 1, -1, -1, 1, -1, 1, -1, 1, 1, -1, 1, -1, -1, 1, -1, 1, -1, 1, -1, -1, 1, 1, 1, -1, 1, -1, 1, 1, 1, -1, -1, 1, 1, -1, 1, 1, 1, -1, -1, 1, 1, 1, 1, -1, 1, 1, -1, -1, 1, 1, 1, -1, 1, -1, -1, 1, 1, -1, -1, -1, 1, 1, 1, -1, 1, 1, 1, -1, -1, -1, 1, -1, -1, 1, 1, -1, -1, -1, 1, 1, -1, 1, 1, -1, 1, -1, 1, 1, -1, -1, 1, 1, -1, 1, -1, 1, -1, 1, -1, 1, 1, -1, -1, 1, -1, -1, 1, -1, -1, 1, -1, 1, -1, -1, 1, -1, -1, -1, 1, -1, 1, -1, -1, -1, 1, 1, -1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, -1, 1, 1, 1, 1, 1, -1, 1, 1, 1, -1, -1, 1, -1, -1, -1, -1, -1, 1, -1, 1, 1, 1, 1, 1, -1, 1, 1, -1, 1, -1, 1, -1, 